# Working with Redis for Caching

This notebook demonstrates how to use the `RedisCache` and `RedisSemanticCache` classes from the langchain-redis package to implement caching for LLM responses using Redis.

## Installation

In [ ]:
%pip install ipywidgets
%pip install langchain-core
%pip install langchain-redis
%pip install langchain-openai
%pip install redis

## Importing Required Libraries

In [ ]:
import os
import time
import redis

from langchain.globals import set_llm_cache
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_redis import RedisCache, RedisSemanticCache

## Setting up Redis Connection

In [ ]:
REDIS_URL = "redis://localhost:6379"
redis_client = redis.from_url(REDIS_URL)
redis_client.ping()

## Set the OpenAI API key

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

## Using Redis as a Standard Cache

In [ ]:
redis_cache = RedisCache(redis_url=REDIS_URL)
set_llm_cache(redis_cache)

llm = OpenAI(temperature=0)

def execute_with_timing(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time

# First call (not cached)
prompt = "Explain the concept of caching in three sentences."
result1, time1 = execute_with_timing(prompt)
print(f"First call (not cached):")
print(f"{result1}\nTime: {time1:.2f} seconds\n")

# Second call (should be cached)
result2, time2 = execute_with_timing(prompt)
print(f"Second call (cached):")
print(f"{result2}\nTime: {time2:.2f} seconds\n")

print(f"Speed improvement: {time1 / time2:.2f}x faster")

# Clear the cache
redis_cache.clear()
print("Cache cleared")

## Using Redis as a Semantic Cache

In [ ]:
embeddings = OpenAIEmbeddings()
semantic_cache = RedisSemanticCache(
    redis_url=REDIS_URL, embeddings=embeddings, distance_threshold=0.2
)

set_llm_cache(semantic_cache)

# Original prompt
original_prompt = "What is the capital of France?"
result1, time1 = execute_with_timing(original_prompt)
print(f"Original query:\nPrompt: {original_prompt}\n")
print(f"{result1}\nTime: {time1:.2f} seconds\n")

# Semantically similar prompt
similar_prompt = "Can you tell me the capital city of France?"
result2, time2 = execute_with_timing(similar_prompt)
print(f"Similar query:\nPrompt: {similar_prompt}\n")
print(f"{result2}\nTime: {time2:.2f} seconds\n")

print(f"Speed improvement: {time1 / time2:.2f}x faster")

## Cleanup

In [ ]:
# Clear the semantic cache
semantic_cache.clear()
print("Semantic cache cleared")